In [1]:
import time
import math
import imutils
import os
import sys
import numpy as np
import cv2
#sys.path.append(os.path.dirname(__file__) + "/../")
from scipy.misc import imread, imsave
from skimage.measure import structural_similarity as ssim
from config import load_config
from dataset.factory import create as create_dataset
from nnet import predict
from util import visualize
import cv2
from dataset.pose_dataset import data_to_input


from multiperson.detections import extract_detections
from multiperson.predict import SpatialModel, eval_graph, get_person_conf_multicut
from multiperson.visualize import PersonDraw, visualize_detections
import matplotlib.pyplot as plt
import tensorflow as tf
from tkinter import messagebox

In [2]:
def vibrate():
    pass
def compare_images(slope1, slope2, allowance):
    for key in slope1:
        if abs(slope1[key]-slope2[key]) > allowance:
            vibrate(key)
            print("error at : ", key)
            
            
def slope_calc(co1):
    body_dict={'nose_right': co1[0],
               'nose_left': co1[1],
              'right_eye_ear': co1[2],
              'left_eye_ear': co1[3],
              'right_upper_arm':co1[4],
              'left_upper_arm':co1[5],
              'right_forearm': co1[6],
              'left_forearm': co1[7],
               'right_upper_leg':co1[8],
               'left_upper_leg':co1[9],
               'right_shin':co1[10],
               'left_shin':co1[11]
              }
    body_dict['backbone']=[(int((body_dict['right_upper_arm'][0][0]+body_dict['left_upper_arm'][0][0])/2),
                           int((body_dict['right_upper_arm'][0][1]+body_dict['left_upper_arm'][0][1])/2)),
                           (int((body_dict['right_upper_leg'][0][0]+body_dict['left_upper_leg'][0][0])/2),
                           int((body_dict['right_upper_leg'][0][1]+body_dict['left_upper_leg'][0][1])/2))]
    for key in body_dict:
        a=math.atan((body_dict['backbone'][1][1]-body_dict['backbone'][0][1])/(body_dict['backbone'][0][0]-body_dict['backbone'][1][0]))
        slopes[key]=(math.atan((body_dict[key][1][1]-body_dict[key][0][1])/(body_dict[key][0][0]-body_dict[key][1][0])))-a
    return slopes

In [30]:
def run_predict(frame):

    # Load and setup CNN part detector
    tf.reset_default_graph()
    image= frame
    image_batch = data_to_input(frame)
    
    # Compute prediction_n with the CNN
    outputs_np = sess.run(outputs, feed_dict={inputs: image_batch})
    scmap, locref, pairwise_diff = predict.extract_cnn_output(outputs_np, cfg, dataset.pairwise_stats)
    detections = extract_detections(cfg, scmap, locref, pairwise_diff)
    unLab, pos_array, unary_array, pwidx_array, pw_array = eval_graph(sm, detections)
    m=time.time()
    person_conf_multi = get_person_conf_multicut(sm, unLab, unary_array, pos_array)
    img = np.copy(image)
    #coor = PersonDraw.draw()
    visim_multi = img.copy()
    co1=draw_multi.draw(visim_multi, dataset, person_conf_multi, image)
    return pos_array

In [8]:
start_time=time.time()
cfg = load_config("demo/pose_cfg_multi.yaml")
dataset=create_dataset(cfg)
sm = SpatialModel(cfg)
sm.load()
tf.reset_default_graph()
draw_multi = PersonDraw()
sess, inputs, outputs = predict.setup_pose_prediction(cfg)
fps_time=0
# Read image from file
slopes={}
dir=os.listdir("stick")
k=0
cap=cv2.VideoCapture('exer.mp4')
cap_user=cv2.VideoCapture(0)
i=0
while (True):
    ret, orig_frame= cap.read()
    ret2, orig_frame_user= cap_user.read()
    if i%25 == 0:                   
        
        frame = cv2.resize(orig_frame, (0, 0), fx=0.50, fy=0.50)
        #frame=orig_frame
        user_frame=cv2.resize(orig_frame_user, (0, 0), fx=0.50, fy=0.50)
        co1=run_predict(frame)
        user_co1=run_predict(user_frame)
        
        
        try:
            slope_reqd=slope_calc(co1)
            slope_user=slope_calc(user_co1)
            compare_images(slope_reqd, slope_user)
        except IndexError:
            #if len(co1)!=len(user_co1):
            pass
            
        cv2.putText(user_frame,
                    "FPS: %f" % (1.0 / (time.time() - fps_time)),
                    (10, 10),  cv2.FONT_HERSHEY_SIMPLEX, 0.5,
                    (0, 255, 0), 2)
        
        cv2.imshow('user_frame', user_frame)
        cv2.imshow('frame', frame)
        fps_time=time.time()
        #visualize.waitforbuttonpress()
        if cv2.waitKey(10)==ord('q'):
            break
elapsed= time.time()-start_time
#print("sse score : ", sse)
cap.release()
cap_user.release()
cv2.destroyAllWindows()

INFO:tensorflow:Restoring parameters from models/coco/coco-resnet-101


INFO:tensorflow:Restoring parameters from models/coco/coco-resnet-101


num_people:  2
#tracked objects: 0
num_people:  1
#tracked objects: 1
num_people:  1
#tracked objects: 1
num_people:  1
#tracked objects: 1
num_people:  3
#tracked objects: 1
num_people:  1
#tracked objects: 1
num_people:  3
#tracked objects: 1
num_people:  1
#tracked objects: 1
num_people:  5
#tracked objects: 1
num_people:  1
#tracked objects: 1


In [10]:
pwidx_array

NameError: name 'pwidx_array' is not defined

In [7]:
cap.release()
cap_user.release()
cv2.destroyAllWindows()

In [12]:
image= frame
image_batch = data_to_input(frame)

# Compute prediction_n with the CNN
outputs_np = sess.run(outputs, feed_dict={inputs: image_batch})
scmap, locref, pairwise_diff = predict.extract_cnn_output(outputs_np, cfg, dataset.pairwise_stats)
detections = extract_detections(cfg, scmap, locref, pairwise_diff)
unLab, pos_array, unary_array, pwidx_array, pw_array = eval_graph(sm, detections)
m=time.time()
person_conf_multi = get_person_conf_multicut(sm, unLab, unary_array, pos_array)
img = np.copy(image)
#coor = PersonDraw.draw()
visim_multi = img.copy()
co1=draw_multi.draw2(visim_multi, dataset, person_conf_multi, image)

num_people:  5
#tracked objects: 1


In [27]:
pos_array1=pos_array.round().astype(int)

In [28]:
pos_array1

array([[300,  87],
       [306,  83],
       [300,  84],
       [320,  86],
       [337, 108],
       [301, 102],
       [364,  69],
       [355, 132],
       [291,  76],
       [354,  21],
       [295,  38],
       [353, 130],
       [295,  40],
       [354,  21],
       [335, 202],
       [305, 204],
       [349, 267],
       [288, 251],
       [396, 322],
       [290, 293]])

In [29]:
pos_array

array([[299.80378212,  86.60445786],
       [305.66500473,  83.29327565],
       [299.62593976,  84.40048602],
       [320.23118019,  86.29758811],
       [336.91520309, 107.66185451],
       [300.94690591, 101.98846591],
       [364.30259404,  69.32882512],
       [354.9719398 , 131.80669855],
       [290.6085434 ,  75.84775499],
       [354.22004008,  20.73260653],
       [295.01453686,  38.40702653],
       [352.56647182, 130.39948952],
       [294.56543016,  39.7606442 ],
       [354.24010086,  21.42583728],
       [335.01036382, 201.75517106],
       [305.25822306, 203.57265311],
       [348.92772913, 266.89176798],
       [287.72134924, 250.79992747],
       [395.92394735, 322.20143902],
       [290.40567827, 292.94609219]])